In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import numpy as np
nb_dir = '/home/alon/Repos/'
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path
#import Utilities as utl
import matplotlib.pyplot as plt

In [2]:
fpath = '/bigstore/Images2019/Jen/NFkBDynamics/TNFTitr_HighMOI_Dec122019_2019Dec12/acq_2'
from oyLabCode import Metadata
MD = Metadata(fpath)

loaded Metadata from pickle file


In [82]:
import time
t = time.time()
MD.CalculateDriftCorrection(MD.posnames[2])
time.time()-t

opening img_D02_000000145_000000144_DeepBlue_000_001.tif
calculating drift correction for position D02

calculated drift correction for position D02
saved metadata


52.643532276153564

In [85]:
img = MD.stkread(Position=MD.posnames[2],Channel='DeepBlue',register=False)


opening img_D02_000000145_000000144_DeepBlue_000_001.tif

In [86]:
import napari
with napari.gui_qt():
    viewer = napari.view_image(img, rgb=False)

In [4]:
img.shape

(145, 2048, 2448)

In [4]:
from scipy.signal import fftconvolve

DataPost = img[1:,:,:]
DataPre = img[:-1,:,:]

        ##this is in prep for # Zs>1
        #DataPre = np.reshape(DataPre,(DataPre.shape[1],DataPre.shape[2],len(ZsToLoad), len(frames)-1));
        #DataPost = np.reshape(DataPost,(DataPost.shape[1],DataPost.shape[2],len(ZsToLoad), len(frames)-1));

In [6]:
from pyfftw.interfaces.numpy_fft import fft2, ifft2

In [11]:
import time
t=time.time()
img_fft_1 = np.fft.fftshift(fft2(DataPre-np.mean(DataPre,axis=(1,2),keepdims=True),axes=(1,2)),axes=(1,2))
img_fft_2 = np.fft.fftshift(fft2(np.rot90(DataPost-np.mean(DataPost,axis=(1,2),keepdims=True),axes=(1, 2),k=2),axes=(1,2)),axes=(1,2))
imXcorr = np.abs(np.fft.ifftshift(ifft2(img_fft_1*img_fft_2,axes=(1,2))))
print(time.time()-t)

In [13]:
import time
A = (DataPre-np.mean(DataPre,axis=(1,2),keepdims=True))
B = np.rot90(DataPost-np.mean(DataPost,axis=(1,2),keepdims=True),axes=(1, 2),k=2)

t=time.time()
img_fft_1 = fft2(A,axes=(1,2))
img_fft_2 = fft2(B,axes=(1,2))
imXcorr = np.abs(np.fft.ifftshift(ifft2(img_fft_1*img_fft_2,axes=(1,2))))
print(time.time()-t)

141.09859704971313


In [14]:
import napari
with napari.gui_qt():
    viewer = napari.view_image(imXcorr, rgb=False)

In [7]:
#calculate cross correlation
imXcorr = fftconvolve((DataPre-np.mean(DataPre,axis=(1,2),keepdims=True)), np.rot90(DataPost-np.mean(DataPost,axis=(1,2),keepdims=True),axes=(1, 2),k=2), mode='same', axes=[1,2])

In [72]:
from pyfftw.interfaces.numpy_fft import rfft2, irfft2
        
frames = MD.frames
ZsToLoad=[1]
pos = MD.posnames[2]
DataPre = MD.stkread(Position=MD.posnames[2],Channel='DeepBlue',register=False)
print('\ncalculating drift correction for position ' + pos)

DataPre = DataPre-np.mean(DataPre,axis=(1,2),keepdims=True)
                                  
DataPost = DataPre[1:,:,:].transpose((1,2,0))
DataPre = DataPre[:-1,:,:].transpose((1,2,0))

#this is in prep for # Zs>1
DataPre = np.reshape(DataPre,(DataPre.shape[0],DataPre.shape[1],len(ZsToLoad), len(frames)-1));
DataPost = np.reshape(DataPost,(DataPost.shape[0],DataPost.shape[1],len(ZsToLoad), len(frames)-1));

#calculate cross correlation
#imXcorr = fftconvolve((DataPre-np.mean(DataPre,axis=(0,1),keepdims=True)), np.rot90(DataPost-np.mean(DataPost,axis=(0,1),keepdims=True),k=2), mode='same', axes=[0,1])

DataPost = np.rot90(DataPost,axes=(0, 1),k=2)
        
imXcorr = np.zeros_like(DataPre)
for i in np.arange(DataPre.shape[-1]):
    img_fft_1 = rfft2(DataPre[:,:,:,i],axes=(0,1))
    img_fft_2 = rfft2(DataPost[:,:,:,i],axes=(0,1))
    imXcorr[:,:,:,i] = np.abs(irfft2(img_fft_1*img_fft_2,axes=(0,1)))
        
        

opening img_D02_000000145_000000144_DeepBlue_000_001.tif
calculating drift correction for position D02


In [68]:
t = time.time()
img_fft_1 = rfft2(DataPre,axes=(0,1))
time.time()-t

29.612560510635376

In [70]:
t = time.time()
for i in np.arange(DataPre.shape[-1]):
    img_fft_1 = rfft2(DataPre[:,:,:,i],axes=(0,1))
print(time.time()-t)

16.61125922203064


In [47]:
np.arange(DataPre.shape[-1],0,-1)

array([144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132,
       131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119,
       118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106,
       105, 104, 103, 102, 101, 100,  99,  98,  97,  96,  95,  94,  93,
        92,  91,  90,  89,  88,  87,  86,  85,  84,  83,  82,  81,  80,
        79,  78,  77,  76,  75,  74,  73,  72,  71,  70,  69,  68,  67,
        66,  65,  64,  63,  62,  61,  60,  59,  58,  57,  56,  55,  54,
        53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,  42,  41,
        40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,  29,  28,
        27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,  15,
        14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,   2,
         1])

In [59]:
import napari
with napari.gui_qt():
    viewer = napari.view_image(np.squeeze(imXcorr), rgb=False)

In [ ]:
 
        imXcorr = np.zeros_like(DataPre)
        for i in np.arange(DataPre.shape[-1]):
            print(i)
            img_fft_1 = fft2(np.squeeze(DataPre[:,:,:,i]),axes=(0,1))
            img_fft_2 = fft2(np.squeeze(DataPost[:,:,:,i]),axes=(0,1))
            imXcorr[:,:,:,i] = np.abs(np.fft.ifftshift(ifft2(img_fft_1*img_fft_2,axes=(0,1))))
        
        
        
        #if more than 1 slice is calculated, look for mean shift
        imXcorrMeanZ = np.mean(imXcorr,axis=2)
        c = []
        for i in range(imXcorrMeanZ.shape[-1]):
            c.append(np.squeeze(imXcorrMeanZ[:,:,i]).argmax())

        d = np.transpose(np.unravel_index(c, np.squeeze(imXcorrMeanZ[:,:,0]).shape))-np.array(np.squeeze(imXcorrMeanZ[:,:,0]).shape)/2
        D = np.insert(np.cumsum(d, axis=0), 0, [0,0], axis=0)